In [27]:
import pandas as pd
from apify_client import ApifyClient
from datetime import datetime


apify_client = ApifyClient('APIFY_KEY')

In [28]:
try:
    # Fetch details of the actor
    actor = apify_client.actor('epctex/google-jobs-scraper')
    actor_info = actor.get()
    print("Actor Info:", actor_info)
except Exception as e:
    print("Error:", e)

Actor Info: {'id': 'nopnOEWIYjLQfBqEO', 'userId': '3qhAcS3ZRJK4a7uNG', 'name': 'google-jobs-scraper', 'username': 'epctex', 'description': 'The most comprehensive Google Jobs Scraper ever! Extremely configurable, highly customizable, and blazing fast. Retrieve salary, social links, apply links, and all detailed sections. Both for data retrieval and API Integration. Easy use without any limits!', 'restartOnError': False, 'isPublic': True, 'createdAt': datetime.datetime(2023, 12, 26, 9, 19, 41, 105000, tzinfo=datetime.timezone.utc), 'modifiedAt': datetime.datetime(2024, 12, 9, 0, 41, 25, 95000, tzinfo=datetime.timezone.utc), 'stats': {'totalBuilds': 294, 'totalRuns': 71327, 'totalUsers': 247, 'totalUsers7Days': 37, 'totalUsers30Days': 73, 'totalUsers90Days': 135, 'lastRunStartedAt': datetime.datetime(2024, 12, 9, 4, 4, 33, 152000, tzinfo=datetime.timezone.utc), 'publicActorRunStats30Days': {'ABORTED': 18, 'FAILED': 4, 'SUCCEEDED': 13379, 'TIMED-OUT': 454, 'TOTAL': 13855}}, 'versions': []

In [29]:
# Start an actor and wait for it to finish
position_df = pd.DataFrame()
job_titles = ["Data Analyst", "Data Engineer", "Data Scientist", "Software Developer", "Product Manager", "Digital Marketer", "Machine Learning Engineer"]
locations = {"w+CAIQICIHVG9yb250bw==": "Toronto", "w+CAIQICIJVmFuY291dmVy": "Vancouver"} 

for uule in locations:
    print(locations[uule])
    for jt in job_titles:
        print(jt)
        run_input = {
            "csvFriendlyOutput": True,
            "includeUnfilteredResults": False,
            "maxConcurrency": 10,
            "maxPagesPerQuery": 3,
            "queries": f"https://www.google.com/search?ibp=htl;jobs&q={jt}&uule={uule}",
            "saveHtml": False,
            "saveHtmlToKeyValueStore": False,
        }

        actor_call = apify_client.actor(
            'dan.scraper/google-jobs-scraper').call(run_input=run_input)

        dataset_items = apify_client.dataset(
            actor_call['defaultDatasetId']).list_items().items

        d = pd.DataFrame(dataset_items)
        d["query"] = jt
        d["location"] = locations[uule]
        d["run_time"] = str(datetime.now())

        position_df = pd.concat([position_df, d])

    print("="*30)
    # break # Only Run For Toronto

Toronto
Data Analyst
Data Engineer
Data Scientist
Software Developer
Product Manager
Digital Marketer
Machine Learning Engineer
Vancouver
Data Analyst
Data Engineer
Data Scientist
Software Developer
Product Manager
Digital Marketer
Machine Learning Engineer


In [30]:
position_df.groupby("location")["query"].value_counts()

location   query                    
Toronto    Data Analyst                 10
           Data Scientist               10
           Machine Learning Engineer    10
           Product Manager              10
Vancouver  Data Analyst                 10
           Digital Marketer             10
           Product Manager              10
Name: count, dtype: int64

In [32]:
#_ = position_df.pop("thumbnail") # not useful, with na values
position_df.to_csv("data/raw_google_1130.csv", index=False)


In [37]:
position_df.head(1)
# print(position_df.iloc[0]['description'])

,title,companyName,location,via,shareLink,thumbnail,metadata,description,applyOptions,query,run_time,jobHighlights,extras
0,Business / Data Analyst,TEKsystems,Toronto,TEKsystems Careers,https://www.google.com/search?ibp=htl;jobs&q=D...,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",{'salary': 'CA$43.87–CA$50.37 an hour'},Description\n\nOur clients Data Analytics team...,"[{'title': 'TEKsystems Careers', 'link': 'http...",Data Analyst,2024-12-08 21:05:18.608421,NaN,NaN


In [38]:
from opensearchpy import OpenSearch, helpers

In [2]:
host = 'search-swift-hire-dev-jfmldmym4cfbiwdhwmtuqq6ihy.us-west-2.es.amazonaws.com'
port = 443
auth = ('swift', 'Hire123!') # For testing only. Don't store credentials in code.

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [4]:
indices = client.indices.get_alias('*')
print(indices)


{'.kibana_109854227_swift_1': {'aliases': {'.kibana_109854227_swift': {}}}, 'jobmatch_resume_dev_wei': {'aliases': {}}, '.ql-datasources': {'aliases': {}}, 'swift_dev_felix_kelly': {'aliases': {}}, 'swift_dev_felix_kelly_resume': {'aliases': {}}, 'swift_dev_felix_kelly_jobposts': {'aliases': {}}, '.opendistro-reports-definitions': {'aliases': {}}, 'swift_dev_new': {'aliases': {}}, 'weilun_swift_demo': {'aliases': {}}, '.opendistro-reports-instances': {'aliases': {}}, '.opendistro_security': {'aliases': {}}, 'job_ocr_results': {'aliases': {}}, 'scrape_test': {'aliases': {}}, 'job_postings': {'aliases': {}}, '.opensearch-observability': {'aliases': {}}, 'jobmatch_dev_wei_updated': {'aliases': {}}, '.opensearch-sap-log-types-config': {'aliases': {}}, 'demo_job_seekers': {'aliases': {}}, 'swift_dev': {'aliases': {}}, 'opensearch_dashboards_sample_data_ecommerce': {'aliases': {}}, 'btang-resume-index': {'aliases': {}}, 'swifthire_users_prod': {'aliases': {}}, 'jobs_shiyi&xiner': {'aliases':

In [24]:
index_name = 'swifthire_jobs_dev'  # Replace with your index

response = client.search(
    index=index_name,
    # body={
    #     "query": {
    #         # "match": {
    #         #     "searched_job_title": "Data Scientist"
    #         # }
    #     }
    # }
)
# response = client.search(
#     index=index_name,
#     body={
#         "size": 10,
#         "query": {
#             "bool": {
#                 "must": [
#                     { "match": {"query": "Data Scientist"}}
#                     # { "match": {"location": "Toronto"}}
#                 ]
#             }
#         }
#     }
# )

In [26]:
print(len(hits))

10


In [25]:
hits = response['hits']['hits']
for doc in hits:
    print(doc['_source'])  # Access the document content


{'searchQuery': {'term': 'Data Engineer', 'page': 2, 'type': 'SEARCH', 'domain': 'google.com', 'countryCode': 'US', 'languageCode': None, 'locationUule': None, 'resultsPerPage': 10}, 'url': 'https://www.google.com/search?ibp=htl%3Bjobs&q=Data%20Engineer&start=10', 'hasNextPage': True, 'googleJobs': [{'title': 'Data Engineer, Analytics', 'companyName': 'Meta', 'location': '  Raleigh, NC   ', 'via': 'via Meta Careers Jobs', 'description': "Summary:\n\nMeta Platforms, Inc. (Meta), formerly known as Facebook Inc., builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps and services like Messenger, Instagram, and WhatsApp further empowered billions around the world. Now, Meta is moving beyond 2D screens toward immersive experiences like... augmented and virtual reality to help build the next evolution in social technology. To apply, click “Apply to Job” online on this web page.\n\nRequired S

In [20]:
print(response)

{'took': 520, 'timed_out': False, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 2.6568744, 'hits': [{'_index': 'swifthire_jobs_dev', '_id': 'y5R9HZABhNg_xFXvWjxh', '_score': 2.6568744, '_source': {'title': 'Senior Data Scientist - Reinforcement Learning', 'companyName': 'Home Depot / THD', 'location': 'Vancouver', 'via': 'via Glassdoor', 'description': "Position Purpose:\n\nThe Sr. Data Scientist is responsible for leading data science initiatives that drive business profitability, increased efficiencies and improved customer experience. This role assists in the development of the Home Depot advanced analytics infrastructure that informs decision making. Sr. Data Scientists are expected to seek out business opportunities to leverage data science... as a competitive advantage. Based on the specific data science team, this role would need to be Proficient in one or more data science specializations

In [21]:
doc['_source']

{'title': 'Data Scientist/Engineer',
 'companyName': 'University of Chicago (UC)',
 'location': 'Toronto',
 'via': 'via AAAI Career Center',
 'description': "Location: Chicago, IL\n\nJob Description...\n• Develop and build upon open source software for the monitoring of AI in the clinic.\n• Work closely with lab members and data scientists within the health system to support the study of model implementation.\n• Assist in the research, development and/or application of machine learning models to clinical data.\n• Participate in the evaluation and validation of developed models across multiple datasets and/or settings.\n• Support lab efforts to publish research findings in scientific journals and/or present research findings at relevant scientific conferences and seminars.\n• Participate in lab activities including collaborative meetings, offering feedback to other members of the lab, literature review and preparing materials for research presentations.\n• Analyzes moderately complex da

In [22]:
index_name = "swift_dev"

if not client.indices.exists(index_name):
    client.indices.create(index=index_name)

def doc_generator(df):
    for i, row in df.iterrows():
        doc = {
            "_index": index_name,
            "_source": row.to_dict(),
        }
        yield doc

helpers.bulk(client, doc_generator(position_df))

print("Data Saved to ES")

NameError: name 'position_df' is not defined

In [14]:
position_df["query"].value_counts()

query
Data%20Analyst          30
Data%20Scientist        30
Software%20Developer    30
Data Engineer           30
Product Manager         30
Digital%20Marketer      20
Name: count, dtype: int64

In [42]:
# Replace NaN with None (which Elasticsearch will treat as null)
position_df = position_df.where(pd.notnull(position_df), None)


In [43]:
from elasticsearch import Elasticsearch
import pandas as pd

# Initialize Elasticsearch client
es = Elasticsearch(
    ['https://localhost:9200'],
    basic_auth=('elastic', 'elastic')
)

# Convert the DataFrame to a dictionary for each row
records = position_df.to_dict(orient='records')

# Index each record into Elasticsearch
for idx, record in enumerate(records):
    es.index(index='brave_project', id=idx, body=record)

print("Data successfully indexed into Elasticsearch.")


Data successfully indexed into Elasticsearch.
